<!-- Autoheader begin -->
<hr/>
<div id="navtitle_2_1_jl" style="text-align:center; font-size:16px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_1_3_chirp.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_1_3_chirp.ipynb" style="font-size:13px">I.3 Interaction of a Two-Level-System with a Chirped Laser Pulse</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_1_1_TLS.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_1_1_TLS.ipynb" style="font-size:13px">I.1 Population Inversion in a Two-Level-System</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_2_lambda.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method and GRAPE</a><br>
        <a href="jl_exercise_3_1_TLS.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_2_1_TLS.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Population Inversion in a Two-Level-System using Parameter Optimization

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

This notebook contains the code for running an optimization to achieve a
population inversion in a two-level system.
You will learn how to use the `NLopt` package (via `Optimization.jl`) in order to optimize a
pulse shape by tuning the pulse parameters such as the pulse duration and intensity. The goal of the optimization is to drive the two-level system such that all the population is perfectly transferred from the ground to the excited state, similar to [Exercise I.1](jl_exercise_1_1_TLS.ipynb). In this notebook we employ parameter optimization which searches the optimization landscape directly by evaluating the optimization functionals at different points of the parameter space.

## Setup

First, we need to load some of the libraries that we will need throughout
this notebook. There main packages are:
* `QuantumPropagators`: Set up time dependent Hamiltonians and propagate them
* `OrdinaryDiffEq`: The backend to use for `QuantumPropagators.propagate`
* `Optimization` for optimization, with `OptimizationNLopt` providing the connection to the `NLOpt` backend package.

In [ ]:
using QuantumPropagators: hamiltonian, propagate
using OrdinaryDiffEq
using Optimization, OptimizationNLopt

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

In [ ]:
# Some utilities for showing hints and solutions
include(joinpath("utils", "exercise_2_TLS.jl"));

### Defining the Hamiltonian

We begin by defining the Hamiltonian, the guess field as well as the initial and target states.

The Hamiltonian $\op{H}_{0} = - \frac{\omega}{2} \op{\sigma}_{z}$ represents
a simple qubit with energy level splitting $\omega$ in the basis
$\{\ket{0},\ket{1}\}$. The control field $\epsilon(t)$ is assumed to couple
via the Hamiltonian $\op{H}_{1}(t) = \epsilon(t) \op{\sigma}_{x}$ to the
qubit, i.e., the control field effectively drives transitions between both
qubit states.

In this notebook, we will optimize the amplitude $E_0$ and pulse length
$\Delta T = t_f - t_i$ of a pulse with a Blackman shape. For simplicity, we
choose the carrier frequency of the pulse to match the energy splitting of
the qubits. The pulse is then defined as

$$
\epsilon(t) = E_0 \cos(\omega t) B(t; t_i, t_f)
$$

with the Blackman shape defined as

$$
B(t; t_i, t_f) =
\frac{1}{2}\left(
    1 - a - \cos\left(2\pi \frac{t - t_i}{t_f - t_i}\right)
    + a \cos\left(4\pi \frac{t - t_i}{t_f - t_i}\right)
\right)\,, \text{ if } t_i < t < t_f;
\qquad B(t; t_i, t_f) = 0 \text{ otherwise}
$$

with $a = 0.16$. A Blackman shape looks nearly identical to a Gaussian with a
6-sigma interval between $t_i$ and $t_f$. Unlike the Gaussian, however, it
will go exactly to zero at the edges. Thus, Blackman pulses are often
preferable to Gaussians.

In [ ]:
"""
Blackman shaped pulse

# Keyword Arguments

* `ω`: carrier frequency of the pulse
* `E₀`: pulse amplitude
* `ti`: start of the pulse
* `tf`: end of the pulse
* `α`: Blackman parameter, should be `0.16`
"""
function blackman_pulse(t; ω, E₀, ti, tf, a=0.16)
    ΔT = tf - ti
    if (t < ti) || (t > tf)
        return 0.0
    else
        return (E₀/2) * cos(ω*t) * (1 - a - cos(2π * (t-ti)/ΔT) + a * cos(4π * (t-ti)/ΔT))
    end
end


"""
Two-level-system Hamiltonian and canonical eigenstates.

# Arguments

* `ω`: Energy separation of the qubit levels
* `ϵ`: Control function
"""
function ham_and_states(ω, ϵ)
    σz = Float64[1 0; 0 -1]
    σx = Float64[0 1; 1 0]
    H₀ = -(1/2) * ω * σz
    ket0 = ComplexF64[1, 0]
    ket1 = ComplexF64[0, 1]
    return hamiltonian(H₀, (σx, ϵ)), ket0, ket1
end

## Simulating the dynamics with the guess field

Before starting with the actual optimization, we first simulate the
dynamics under a guess field $\epsilon_{0}(t)$. To this end, we need to define
the time grid of the dynamics. We define the initial state to
be at time $t=0$ and consider a total propagation time of $T=10$. The entire
time grid is divided into $5000$ equidistant time steps (corresponding to 5001 time grid
points).

In [ ]:
T = 10.0
nt = 5001

tlist = collect(range(0, T; length=nt));

Next we define the guess pulse itself.
Note that in the following, it is more convenient to use the
pulse length `ΔT = t_f - t_i` instead of `t_i` and `t_f` as
a parameter, since it reduces the number of control parameters by one.

In [ ]:
ω = 12.0  # carrier frequency, should be sufficiently large
E₀ = 0.5  # pulse amplitude
ΔT = 5.0  # pulse length
# make pulse symmetric around the middle of the time interval
ti = T/2 - ΔT/2
tf = T/2 + ΔT/2

ϵ(t) = blackman_pulse(t; ω, E₀, ti, tf)
H, ket0, ket1 = ham_and_states(ω, ϵ);

Then, we solve the equation of motion for the initial state
$\ket{\Psi_{\init}}=\ket{0}$ for the evolution generated by the Hamiltonian $\op{H}(t)$. By passing `storage=true` to the `propagate` function, we will
obtain a matrix `states` that contains the concatenated states at each point
of `tlist`:

In [ ]:
states = propagate(ket0, H, tlist; method=OrdinaryDiffEq, storage=true)

Now we plot the absolute value of the field together with the population of
the two levels as a function of time.

In [ ]:
using QuantumPropagators.Controls: discretize

function plot_population_and_pulse(tlist, states, ϵ)
    E = discretize(ϵ, tlist)
    plot(tlist, abs.(E) / maximum(E); label="|E|", color=:lightgray)
    plot!(tlist, abs2.(states)'; label=["|0⟩" "|1⟩"], ls=[:solid :dash], color=["#1f77b4" "#ff7f0e"])
    plot!(; xlabel="time", ylabel="population", legend=:right)
    infidelity = 1 - abs2.(states[:,end])[2]
    annotate!(
        0, 0.9,
        ("infidelity: $(round(infidelity; digits=5))", 10, :left)
    )
end

plot_population_and_pulse(tlist, states, ϵ)

## Problem 0 - Pulse parameterization

As you can see, the guess pulse does a bad job in transferring the population
from the ground to the excited state. Therefore, we first try to optimize the
pulse parameters by hand to achieve a full population inversion.

We begin by familiarizing ourselves with the pulse parameterization. For the
Blackman shape, `E_0` controls the pulse amplitude, and `ΔT` determines the
length of the pulse. Try changing the arguments of
`evolve_and_plot_parameterized_pulse`, such that you achieve an infidelity
below 0.001.

In [ ]:
function evolve_and_plot_parameterized_pulse(; E₀, ΔT, ω=12.0, T=10, nt=5001)
    ti = T/2 - ΔT/2
    tf = T/2 + ΔT/2
    ϵ(t) = blackman_pulse(t; ω, E₀, ti, tf)
    H, ket0, ket1 = ham_and_states(ω, ϵ)
    tlist = collect(range(0, T; length=nt));
    states = propagate(ket0, H, tlist; method=OrdinaryDiffEq, storage=true)
    plot_population_and_pulse(tlist, states, ϵ)
end

In [ ]:
# problem_0.hint

In [ ]:
# problem_0.solution

## Initialize optimization

Now we can turn towards optimization. To this end, we use the optimization methods provided by the [`NLOpt` package](https://nlopt.readthedocs.io/en/latest/). This package has a [Julia wrapper](https://github.com/JuliaOpt/NLopt.jl) which we could use directly, but there is also a package [Optimization.jl](https://docs.sciml.ai/Optimization/stable/) that provides a common API for defining and solving optimization problems not just via `NLOpt` but also a dozen other optimization toolboxes. For simplicity, we use the well-known Nelder-Mead method. Note, however, that `NLOpt` allows for a wide range of different methods as do the other toolboxes wrapped by Optimization.jl.

The Optimization.jl interface requires to provide a `loss` function, i.e. the optimization functional, which takes a vector `x` of real-valued parameters as well as an object `constants` containing static parameters. This could be another vector, but a [`NamedTuple`](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) is often cleaner. Note that in a different setup, the function to be optimized could be a more general [`OptimizationFunction`](https://docs.sciml.ai/Optimization/stable/API/optimization_function/) object which also contains information on how to calculate gradients. See the [Optimization.jl manual](https://docs.sciml.ai/Optimization/stable/API/optimization_problem/) for details. Since we only study gradient-free optimization in this notebook, we will not need this here.

The `loss` function takes the list `x`, which contains our optimization parameters, i.e., the amplitude $E_{0}$ and the pulse duration $\Delta T$ on input, and returns the infidelity. Specifically, `loss` returns zero if and only if the dynamics obtained from the parameters in the set `x` transfers the initial state $\ket{\Psi(0)}$ perfectly into the target state, i.e.,

$$
\ket{\Psi(0)} =
\ket{0}
\longrightarrow
\ket{\Psi\left(T\right)} =
\ket{1}
$$

In [ ]:
using UnPack: @unpack  # unpack NamedTuple into variables

In [ ]:
function loss(x, constants)
    E₀, ΔT = x
    @unpack ω, T, nt = constants
    ti = T/2 - ΔT/2
    tf = T/2 + ΔT/2
    tlist = collect(range(0, T; length=nt));
    ϵ(t) = blackman_pulse(t; ω, E₀, ti, tf)
    tlist = collect(range(0, T; length=nt));
    H, ket0, ket1 = ham_and_states(ω, ϵ)
    Ψout = propagate(ket0, H, tlist; method=OrdinaryDiffEq)
    fidelity = abs2(Ψout[2])
    return 1.0 - fidelity
end

## Problem 1 - Run optimization

Next, we can run the actual optimization. It requires us to define lower and
upper bounds for all parameters that should be optimized. In our case, we
choose

$$
0 \leq E_{0} \leq 10,
\qquad
0 \leq \Delta T \leq T.
$$

Note that we need to provide guess values for the two parameters. The order
that the parameters should appear in is as follows

* `E_0`
* `ΔT`

The choice of the guess values will often have an appreciable impact on the general success of the optimized solution and can even affect its form since many optimization problems allow for many different solutions.
Your task is now to fill in the upper optimization bounds and to try different
guesses to evaluate their impact on the optimization.

In [ ]:
guess = [#= insert guess parameters here =#]
upper_bounds = [#= insert upper bounds here =#]

In [ ]:
# problem_1.hint

In [ ]:
# problem_1.solution

In [ ]:
prob = OptimizationProblem(
    loss,
    guess,
    (; ω, T, nt),  # this is a NamedTuple, forwarding the global variables
    lb=[0.0, 0.0],
    ub=upper_bounds,
    stopval=(1-0.999),  # below which error to stop the optimization
);

We can check the quality of the guess pulse:

In [ ]:
loss(guess, (; ω, T, nt))

In the optimization, we often want to keep track of the fidelity after each iteration. To this end, we define a ["callback" function](https://docs.sciml.ai/Optimization/stable/API/solve/#CommonSolve.solve-Tuple%7BOptimizationProblem,%20Any%7D) which the optimizer will execute after each step.

In [ ]:
obtained_fidelities = Float64[];  # for keeping track of the fidelity in each iteration

In [ ]:
function callback(state, loss_val)
    global obtained_fidelities
    fid = 1 - loss_val
    push!(obtained_fidelities, fid)
    print("Iteration: $(length(obtained_fidelities)), current fidelity $(round(fid; digits=4))\r")
    return false
end

Lastly, we call `Optimization.solve` to run the optimization:

In [ ]:
obtained_fidelities = Float64[];
res = Optimization.solve(prob, NLopt.LN_NELDERMEAD(); maxiters=500, callback)
println("\n\nHighest fidelity reached: $(round((1 - res.objective) * 100; digits=2))%")
if res.objective < 1e-3
    println("\tcongratulations, you have obtained population inversion!")
else
    println("\tbad guess, please try again!")
end

In [ ]:
plot(obtained_fidelities; marker=:cross, label="", xlabel="optimization iteration", ylabel="fidelity")

## Analyze optimization results

After the optimization we can verify the optimization result by plotting the
pulse as well as the resulting dynamics.

In [ ]:
evolve_and_plot_parameterized_pulse(E₀=res.u[1], ΔT=res.u[2])
println("E₀ = $(round(res.u[1]; digits=3))\nΔT = $(round(res.u[2]; digits=3))")

Try to vary the guess parameters and obtain different solutions. Can
you obtain a pulse which only brings half the population to the excited state? Or a pulse which makes the population go to the excited state and then perfectly back to the ground state again?

It turns out that for these optimization tasks in this problem there is an infinite amount of solutions.
This is because already a single parameter would be sufficient to
achieve the target objective and we over-parameterize the problem with
two parameters. However, in an actual experiment, there are often additional
constraints, e.g., on the maximum amplitude. Set the upper bound for
$E_0$ to $2.0$ (make sure to set the guess value a bit below that).
Can you still obtain solutions with $\Delta T <3.5$?

## Next steps

To continue exploring the optimization of parameterized pulses with NLOpt in a more complex system, you can proceed with [Exercise II.2](jl_exercise_2_2_lambda.ipynb) for the optimization of STIRAP in a three-level system.
Alternatively, if you are interested in optimization with a gradient-based approach, we recommend to have a look at [Exercise III.1](jl_exercise_3_1_TLS.ipynb) in which Krotov's method is used for the opimization you studied in this notebook.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_2_1_jl)
<!-- Autofooter end -->